# Pydantic を使ってみる

In [1]:
from datetime import date
from pathlib import Path

# デフォルトでは Pydantic はインストールされていない。
# pip を使ってインストールする必要がある。
# cf. https://docs.pydantic.dev/latest/install/
from pydantic import BaseModel

## ファイルの情報を定義

In [2]:
class User(BaseModel):
    """ユーザ情報。

    Attributes:
        name (str): ユーザ名。
        birthday (date): 誕生日。
    """

    name: str
    birthday: date


class Metadata(BaseModel):
    """メタデータ。

    Attributes:
        auther (str): 作者。
        version (str): バージョン。
    """

    author: str
    version: str


class UserDatabase(BaseModel):
    """ユーザ情報のデータベース。

    Attributes:
        metadata (Metadata): メタデータ。
        users (list[User]): ユーザ情報の辞書。キーはユーザ名。
    """

    metadata: Metadata
    users: list[User]

## 使用例

In [3]:
json_path = Path.cwd().parent.joinpath("data").joinpath("sample.json")

# data/sample.json を読み込む。
user_database = UserDatabase.model_validate_json(json_path.read_text())

In [4]:
# メタデータを表示する。
user_database.metadata

Metadata(author='nao2c4', version='1.0.0')

In [5]:
# ユーザ情報を表示する。
# 最初の 10 件のみ表示する。
user_database.users[:10]

[User(name='lqhkirijwf', birthday=datetime.date(1988, 11, 27)),
 User(name='apbgjmiagn', birthday=datetime.date(1974, 11, 20)),
 User(name='xxuiavoybm', birthday=datetime.date(2005, 9, 30)),
 User(name='oqtsaakclt', birthday=datetime.date(1905, 8, 17)),
 User(name='jrejrrcnbg', birthday=datetime.date(1919, 3, 26)),
 User(name='kxfpuysezo', birthday=datetime.date(1993, 9, 8)),
 User(name='aqwzoalvdn', birthday=datetime.date(1936, 12, 21)),
 User(name='etchmgxbno', birthday=datetime.date(1994, 8, 25)),
 User(name='qtdrmclyvc', birthday=datetime.date(2003, 1, 23)),
 User(name='lutrhkkbze', birthday=datetime.date(1967, 9, 9))]

In [6]:
# 誕生日でフィルタリングする。
selected_users = [user for user in user_database.users if user.birthday >= date(2000, 1, 1)]
selected_users[:10]

[User(name='xxuiavoybm', birthday=datetime.date(2005, 9, 30)),
 User(name='qtdrmclyvc', birthday=datetime.date(2003, 1, 23)),
 User(name='idvnauiwot', birthday=datetime.date(2002, 12, 14)),
 User(name='bdhivhlugg', birthday=datetime.date(2009, 4, 10)),
 User(name='elgouvuaoq', birthday=datetime.date(2009, 4, 14)),
 User(name='xqkzkbunrw', birthday=datetime.date(2006, 6, 11)),
 User(name='buvauzjlcy', birthday=datetime.date(2008, 5, 15)),
 User(name='netkgnetaf', birthday=datetime.date(2007, 1, 17)),
 User(name='kqlplrjptx', birthday=datetime.date(2000, 12, 23)),
 User(name='upfmcgulri', birthday=datetime.date(2003, 12, 5))]

In [7]:
# ユーザ情報を data/output_pydantic.json に保存する。
output_path = json_path.parent.joinpath("output_pydantic.json")
output_path.write_text(user_database.model_dump_json(indent=4))

1840103

## 時間計測

In [8]:
%%timeit
# 読み込み時間を計測する。
# Pydantic はデータの検証を行うため、読み込みに時間がかかる。
# 言い換えれば、変なデータが入っているとエラーで止まるので、安全性が高い。
# この環境では、使わない場合と比べて倍程度の時間がかかった。
user_database = UserDatabase.model_validate_json(json_path.read_text())

31 ms ± 946 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
# フィルタリングの時間を計測する。
# 読み込み以外は時間は変わらなかった。
selected_users = [user for user in user_database.users if user.birthday < date(2000, 1, 1)]

2.51 ms ± 36.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
